<a href="https://colab.research.google.com/github/Nukte/Turkish_Sentence_Mood_Analysis/blob/main/SpamDetectorCaseStudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Spam Detector | Case Study**

# *Birinci Deneme - Naive Bayes*


*   Naive Bayes (Kelime) Whatsapp gurubundan aldığım kodlar..
*   Naive Bayes (Karakter) Whatsapp gurubundan aldığım kodlarda yaptığım iyleştirme.



## ----- Naive Bayes (Kelime) // Whatsapp Grubundan aldığım kodlar.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
# ---------------------------------------------------------
# A. VERİ HAZIRLIĞI (DATA PREPARATION)
# ---------------------------------------------------------
# 1. Veriyi Yükle
# 'mail_data.csv' dosyasını okuyoruz.
raw_mail_data = pd.read_csv('mail_data.csv')

# 2. Eksik Verileri Doldur
# Eğer boş hücre varsa onları boş string ile dolduruyoruz hata vermemesi için.
mail_data = raw_mail_data.where((pd.notnull(raw_mail_data)),'')

# 3. Etiketleme (Label Encoding)
# Makine kelimeden anlamaz, bu yüzden:
# spam -> 1 (Tehlikeli)
# ham  -> 0 (Güvenli) olarak işaretliyoruz.
mail_data.loc[mail_data['Category'] == 'spam', 'Category'] = 1
mail_data.loc[mail_data['Category'] == 'ham', 'Category'] = 0

X = mail_data['Message']
Y = mail_data['Category'].astype(int)
# ---------------------------------------------------------
# B. NETLEŞTİRME VE TWEAKING (STRATEJİK BÖLÜM)
# ---------------------------------------------------------

# 4. Veriyi Bölme (Stratified Split)
# Veriyi %80 Eğitim (Train), %20 Test olarak ayırıyoruz.
# ÖNEMLİ TWEAK: 'stratify=Y' komutu ile test setinde de eğitim setinde de
# spam oranının eşit olmasını garantiliyoruz. Böylece model dengesiz öğrenmiyor.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3, stratify=Y)

# 5. Özellik Çıkarımı (Feature Extraction & N-Grams)
# Metinleri sayısal vektörlere dönüştürüyoruz.
# TWEAKING DETAYI: Bu sayede model sadece "kazan" kelimesine değil, "ödül kazan" ikilisine de bakıyor.
# Bu yöntem spam tespitini çok daha keskin (precise) hale getirir.
feature_extraction = TfidfVectorizer(  #Kelime analizi
    min_df=1,
    #analyzer='word' şeklinde de tanımlanabilir.
    stop_words='english',
    lowercase=True, ngram_range=(1, 2)
)

# Eğitim verisine göre sözlüğümüzü oluşturuyoruz ve dönüştürüyoruz.
X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)
# ---------------------------------------------------------
# C. MODELLEME (MODELING)
# ---------------------------------------------------------

# 6. Model Seçimi ve Eğitimi
# Naive Bayes (MultinomialNB) metin sınıflandırmada en hızlı ve etkili yöntemlerden biridir.
model = MultinomialNB()
model.fit(X_train_features, Y_train)

# 7. Test ve Sonuç
prediction_on_test_data = model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)

cm = confusion_matrix(Y_test, prediction_on_test_data)

print(f" (Naive Bayes): {accuracy_on_test_data * 100:.2f}%")
print(f"Confusion Matrix:\n{cm}")

# ----------------------------------------------------------------------

# Karmaşıklık Matrisinden değerleri çıkarma
tn = cm[0, 0] # True Negative: Gerçek Ham, Tahmin Ham
fp = cm[0, 1] # False Positive: Gerçek Ham, Tahmin Spam
fn = cm[1, 0] # False Negative: Gerçek Spam, Tahmin Ham
tp = cm[1, 1] # True Positive: Gerçek Spam, Tahmin Spam

gercek_spam = fn + tp
gercek_ham = tn + fp
toplam_test_mail = gercek_spam + gercek_ham

print("\n==================================================================")
print(" NAİVE BAYES KELİME BAZLI ANALİZ SONUCU")
print("==================================================================")

# 1. SPAM Maillerin Durumu (Kaçırılanlar)
print(f"1. GERÇEK SPAM Mail Sayısı: {gercek_spam}")
print(f"   - Tespit Edilen SPAM Maili (TP): {tp}")
print(f"   - Kaçırılan SPAM Maili (FN):   {fn}")

# 2. HAM Maillerin Durumu (Yanlış İşaretlenenler)
print(f"\n2. GERÇEK HAM Mail Sayısı: {gercek_ham}")
print(f"   - Tespit Edilen HAM Maili (TN): {tn}")
print(f"   - SPAM İşaretlenen HAM Maili (FP): {fp}")

print("\n--- ÖZET METRİKLER ---")
print(f"Toplam Test Edilen Mail Sayısı: {toplam_test_mail}")
print(f"Modelin Genel Doğruluğu (Accuracy): {accuracy_on_test_data * 100:.2f}%")
print("==================================================================")


 (Naive Bayes): 95.96%
Confusion Matrix:
[[966   0]
 [ 45 104]]

 NAİVE BAYES KELİME BAZLI ANALİZ SONUCU
1. GERÇEK SPAM Mail Sayısı: 149
   - Tespit Edilen SPAM Maili (TP): 104
   - Kaçırılan SPAM Maili (FN):   45

2. GERÇEK HAM Mail Sayısı: 966
   - Tespit Edilen HAM Maili (TN): 966
   - SPAM İşaretlenen HAM Maili (FP): 0

--- ÖZET METRİKLER ---
Toplam Test Edilen Mail Sayısı: 1115
Modelin Genel Doğruluğu (Accuracy): 95.96%


## -----Naive Bayes (Karakter)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# ---------------------------------------------------------
# A. VERİ HAZIRLIĞI (DATA PREPARATION)
# ---------------------------------------------------------
# 1. Veriyi Yükle
# 'mail_data.csv' dosyasını okuyoruz.
raw_mail_data = pd.read_csv('mail_data.csv')

# 2. Eksik Verileri Doldur
# Eğer boş hücre varsa onları boş string ile dolduruyoruz hata vermemesi için.
mail_data = raw_mail_data.where((pd.notnull(raw_mail_data)),'')

# 3. Etiketleme (Label Encoding)
# Makine kelimeden anlamaz, bu yüzden:
# spam -> 1 (Tehlikeli)
# ham  -> 0 (Güvenli) olarak işaretliyoruz.
mail_data.loc[mail_data['Category'] == 'spam', 'Category'] = 1
mail_data.loc[mail_data['Category'] == 'ham', 'Category'] = 0

X = mail_data['Message']
Y = mail_data['Category'].astype(int)

# ---------------------------------------------------------
# B. NETLEŞTİRME VE TWEAKING (STRATEJİK BÖLÜM)
# ---------------------------------------------------------

# 4. Veriyi Bölme (Stratified Split)
# Veriyi %80 Eğitim (Train), %20 Test olarak ayırıyoruz.
# ÖNEMLİ TWEAK: 'stratify=Y' komutu ile test setinde de eğitim setinde de
# spam oranının eşit olmasını garantiliyoruz. Böylece model dengesiz öğrenmiyor.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3, stratify=Y)

# 5. Özellik Çıkarımı (Feature Extraction & N-Grams)
# Metinleri sayısal vektörlere dönüştürüyoruz.

# TWEAKING DETAYI: 'analyzer='char_wb'' ve 'ngram_range=(2, 2)' parametreleri eklendi.
# Bu sayede model, "kelime" olarak görülemeyen yazım hatalarını (örn: 'fr33', 'w.i.n')
# ve karakter bazlı gizleme tekniklerini yakalar.
feature_extraction = TfidfVectorizer(
    min_df=1,
    analyzer='char_wb',
    lowercase=True,
    ngram_range=(2,2)  # ikili karakter kontrolü  (ab , cd )
)
# Eğitim verisine göre sözlüğümüzü oluşturuyoruz ve dönüştürüyoruz.
X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

# ---------------------------------------------------------
# C. MODELLEME (MODELING)
# ---------------------------------------------------------

# 6. Model Seçimi ve Eğitimi
# Naive Bayes (MultinomialNB) metin sınıflandırmada en hızlı ve etkili yöntemlerden biridir.
model = MultinomialNB()
model.fit(X_train_features, Y_train)

# 7. Test ve Sonuç
prediction_on_test_data = model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)
cm = confusion_matrix(Y_test, prediction_on_test_data)

print(f" (Naive Bayes): {accuracy_on_test_data * 100:.2f}%")
print(f"Confusion Matrix:\n{cm}")

# ----------------------------------------------------------------------

# Karmaşıklık Matrisinden değerleri çıkarma
tn = cm[0, 0] # True Negative: Gerçek Ham, Tahmin Ham
fp = cm[0, 1] # False Positive: Gerçek Ham, Tahmin Spam
fn = cm[1, 0] # False Negative: Gerçek Spam, Tahmin Ham
tp = cm[1, 1] # True Positive: Gerçek Spam, Tahmin Spam

gercek_spam = fn + tp
gercek_ham = tn + fp
toplam_test_mail = gercek_spam + gercek_ham

print("\n==================================================================")
print(" NAİVE BAYES KARAKTER BAZLI ANALİZ SONUCU")
print("==================================================================")

# 1. SPAM Maillerin Durumu (Kaçırılanlar)
print(f"1. GERÇEK SPAM Mail Sayısı: {gercek_spam}")
print(f"   - Tespit Edilen SPAM Maili (TP): {tp}")
print(f"   - Kaçırılan SPAM Maili (FN):   {fn}")

# 2. HAM Maillerin Durumu (Yanlış İşaretlenenler)
print(f"\n2. GERÇEK HAM Mail Sayısı: {gercek_ham}")
print(f"   - Tespit Edilen HAM Maili (TN): {tn}")
print(f"   - SPAM İşaretlenen HAM Maili (FP): {fp}")

print("\n--- ÖZET METRİKLER ---")
print(f"Toplam Test Edilen Mail Sayısı: {toplam_test_mail}")
print(f"Modelin Genel Doğruluğu (Accuracy): {accuracy_on_test_data * 100:.2f}%")
print("==================================================================")

 (Naive Bayes): 97.67%
Confusion Matrix:
[[965   1]
 [ 25 124]]

 NAİVE BAYES KARAKTER BAZLI ANALİZ SONUCU
1. GERÇEK SPAM Mail Sayısı: 149
   - Tespit Edilen SPAM Maili (TP): 124
   - Kaçırılan SPAM Maili (FN):   25

2. GERÇEK HAM Mail Sayısı: 966
   - Tespit Edilen HAM Maili (TN): 965
   - SPAM İşaretlenen HAM Maili (FP): 1

--- ÖZET METRİKLER ---
Toplam Test Edilen Mail Sayısı: 1115
Modelin Genel Doğruluğu (Accuracy): 97.67%


# *İkinci Deneme - Logistic Regression*


*   LR (Kelime)
*   LR (Karakter)




## Logistic Regression (Kelime)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

# ----------------------------------------------------------------------
# A. VERİ YÜKLEME VE ÖN İŞLEME
# ----------------------------------------------------------------------

# 1. Veriyi Yükleme
try:
    df = pd.read_csv('mail_data.csv')
except UnicodeDecodeError:
    df = pd.read_csv('mail_data.csv', encoding='latin-1')

df.columns = ['Category', 'Message']
df = df[['Category', 'Message']]
df = df.where((pd.notnull(df)), '')

# 2. Etiketleme (Label Encoding): bilgisayar anlasın diye ham -> 0, spam -> 1
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

X = df['Message']
Y = df['Category']

# ----------------------------------------------------------------------
# B. VERİYİ BÖLME VE ÖZELLİK ÇIKARIMI
# ----------------------------------------------------------------------

# 3. Veriyi Bölme (Stratified Split)
# Veriyi %80 Eğitim, %20 Test olarak ayırır. stratify=Y, spam oranını korur.
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=0.2,
    random_state=42,
    stratify=Y
)

# 4. Özellik Çıkarımı (TF-IDF N-Gram)
# Metni sayısal vektörlere dönüştürür. ngram_range=(1, 2) ile kelime gruplarını kullanırız.
vectorizer = TfidfVectorizer(
    min_df=1,
    stop_words='english',
    lowercase=True,
    ngram_range=(1, 2)
)

X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

# ----------------------------------------------------------------------
# C. MODELLEME VE DEĞERLENDİRME
# ----------------------------------------------------------------------

# 5. Model Seçimi ve Eğitimi
# Lojistik Regresyon, Naive Bayes'ten daha iyi bir sınıflandırma sınırı çiziyor

model = LogisticRegression(solver='liblinear', C=100, random_state=42)
#liblinear en iyi sonuç veren solver
model.fit(X_train_features, Y_train)

# 6. Test ve Değerlendirme
y_pred = model.predict(X_test_features)
accuracy = accuracy_score(Y_test, y_pred)
cm = confusion_matrix(Y_test, y_pred)

print(f" (Lojistik Regresyon): {accuracy * 100:.2f}%")
print(f"Confusion Matrix:\n{cm}")

# ----------------------------------------------------------------------

# Karmaşıklık Matrisinden değerleri çıkarma
tn = cm[0, 0] # True Negative: Gerçek Ham, Tahmin Ham
fp = cm[0, 1] # False Positive: Gerçek Ham, Tahmin Spam
fn = cm[1, 0] # False Negative: Gerçek Spam, Tahmin Ham
tp = cm[1, 1] # True Positive: Gerçek Spam, Tahmin Spam

gercek_spam = fn + tp
gercek_ham = tn + fp
toplam_test_mail = gercek_spam + gercek_ham

print("\n==================================================================")
print(" LOJİSTİK REGRESYON KELİME BAZLI ANALİZ SONUCU")
print("==================================================================")

# 1. SPAM Maillerin Durumu (Kaçırılanlar)
print(f"1. GERÇEK SPAM Mail Sayısı: {gercek_spam}")
print(f"   - Tespit Edilen SPAM Maili (TP): {tp}")
print(f"   - Kaçırılan SPAM Maili (FN):   {fn}")

# 2. HAM Maillerin Durumu (Yanlış İşaretlenenler)
print(f"\n2. GERÇEK HAM Mail Sayısı: {gercek_ham}")
print(f"   - Tespit Edilen HAM Maili (TN): {tn}")
print(f"   - SPAM İşaretlenen HAM Maili (FP): {fp}")

print("\n--- ÖZET METRİKLER ---")
print(f"Toplam Test Edilen Mail Sayısı: {toplam_test_mail}")
print(f"Modelin Genel Doğruluğu (Accuracy): {accuracy * 100:.2f}%")
print("==================================================================")

 (Lojistik Regresyon): 98.30%
Confusion Matrix:
[[962   4]
 [ 15 134]]

 LOJİSTİK REGRESYON KELİME BAZLI ANALİZ SONUCU
1. GERÇEK SPAM Mail Sayısı: 149
   - Tespit Edilen SPAM Maili (TP): 134
   - Kaçırılan SPAM Maili (FN):   15

2. GERÇEK HAM Mail Sayısı: 966
   - Tespit Edilen HAM Maili (TN): 962
   - SPAM İşaretlenen HAM Maili (FP): 4

--- ÖZET METRİKLER ---
Toplam Test Edilen Mail Sayısı: 1115
Modelin Genel Doğruluğu (Accuracy): 98.30%


## Logistic Regression (Karakter)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

# ----------------------------------------------------------------------
# A. VERİ YÜKLEME VE ÖN İŞLEME
# ----------------------------------------------------------------------

# 1. Veriyi Yükleme
try:
    df = pd.read_csv('mail_data.csv')
except UnicodeDecodeError:
    df = pd.read_csv('mail_data.csv', encoding='latin-1')

df.columns = ['Category', 'Message']
df = df[['Category', 'Message']]
df = df.where((pd.notnull(df)), '')

# 2. Etiketleme (Label Encoding): ham -> 0, spam -> 1
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

X = df['Message']
Y = df['Category']

# ----------------------------------------------------------------------
# B. VERİYİ BÖLME VE ÖZELLİK ÇIKARIMI
# ----------------------------------------------------------------------

# 3. Veriyi Bölme (Stratified Split)
# Veriyi %80 Eğitim, %20 Test olarak ayırır. stratify=Y, spam oranını korur.
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=0.2,
    random_state=42,
    stratify=Y
)

# 4. Özellik Çıkarımı (TF-IDF N-Gram)
# Metni sayısal vektörlere dönüştürür. ngram_range=(1, 2) ile kelime gruplarını kullanırız.
vectorizer = TfidfVectorizer(
    min_df=1,
    analyzer='char_wb',
    lowercase=True,
    ngram_range=(1, 2)
)

X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

# ----------------------------------------------------------------------
# C. MODELLEME VE DEĞERLENDİRME
# ----------------------------------------------------------------------

# 5. Model Seçimi ve Eğitimi
# Lojistik Regresyon, Naive Bayes'ten daha iyi bir sınıflandırma sınırı çizerek FN'i düşürmemizi sağlar.
model = LogisticRegression(solver='liblinear', C=100, random_state=42)
#liblinear ile FP 0 -  FN 13 sonucunu alırken saga ile FP 3 - FN 9 sonucunu alıyorum.
#saga spam mail yakalama konusunda daha iyi fakat normal maillerimizi kaybetmek istemiyorum.
#o yüzden liblinear benim için daha iyi bir seçenek.
model.fit(X_train_features, Y_train)

# 6. Test ve Değerlendirme
y_pred = model.predict(X_test_features)
accuracy = accuracy_score(Y_test, y_pred)
cm = confusion_matrix(Y_test, y_pred)

print(f" (Lojistik Regresyon): {accuracy * 100:.2f}%")
print(f"Confusion Matrix:\n{cm}")

# ----------------------------------------------------------------------

# Karmaşıklık Matrisinden değerleri çıkarma
tn = cm[0, 0] # True Negative: Gerçek Ham, Tahmin Ham
fp = cm[0, 1] # False Positive: Gerçek Ham, Tahmin Spam
fn = cm[1, 0] # False Negative: Gerçek Spam, Tahmin Ham
tp = cm[1, 1] # True Positive: Gerçek Spam, Tahmin Spam

gercek_spam = fn + tp
gercek_ham = tn + fp
toplam_test_mail = gercek_spam + gercek_ham

print("\n==================================================================")
print(" LOJİSTİK REGRESYON KARAKTER BAZLI ANALİZ SONUCU")
print("==================================================================")

# 1. SPAM Maillerin Durumu (Kaçırılanlar)
print(f"1. GERÇEK SPAM Mail Sayısı: {gercek_spam}")
print(f"   - Tespit Edilen SPAM Maili (TP): {tp}")
print(f"   - Kaçırılan SPAM Maili (FN):   {fn}")

# 2. HAM Maillerin Durumu (Yanlış İşaretlenenler)
print(f"\n2. GERÇEK HAM Mail Sayısı: {gercek_ham}")
print(f"   - Tespit Edilen HAM Maili (TN): {tn}")
print(f"   - SPAM İşaretlenen HAM Maili (FP): {fp}")

print("\n--- ÖZET METRİKLER ---")
print(f"Toplam Test Edilen Mail Sayısı: {toplam_test_mail}")
print(f"Modelin Genel Doğruluğu (Accuracy): {accuracy * 100:.2f}%")
print("==================================================================")

 (Lojistik Regresyon): 98.83%
Confusion Matrix:
[[966   0]
 [ 13 136]]

 LOJİSTİK REGRESYON KARAKTER BAZLI ANALİZ SONUCU
1. GERÇEK SPAM Mail Sayısı: 149
   - Tespit Edilen SPAM Maili (TP): 136
   - Kaçırılan SPAM Maili (FN):   13

2. GERÇEK HAM Mail Sayısı: 966
   - Tespit Edilen HAM Maili (TN): 966
   - SPAM İşaretlenen HAM Maili (FP): 0

--- ÖZET METRİKLER ---
Toplam Test Edilen Mail Sayısı: 1115
Modelin Genel Doğruluğu (Accuracy): 98.83%


# *Üçüncü Deneme - LinearSVC*


*   LSVC (Kelime)
*   LSVC (Karakter)



## LSVC (Kelime)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC # En iyi sonuç veren model
from sklearn.metrics import accuracy_score, confusion_matrix

# ----------------------------------------------------------------------
# A. VERİ YÜKLEME VE ÖN İŞLEME
# ----------------------------------------------------------------------

# 1. Veriyi Yükleme
try:
    df = pd.read_csv('mail_data.csv')
except UnicodeDecodeError:
    df = pd.read_csv('mail_data.csv', encoding='latin-1')

df.columns = ['Category', 'Message']
df = df[['Category', 'Message']]
df = df.where((pd.notnull(df)), '')

# 2. Etiketleme: ham -> 0, spam -> 1
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

X = df['Message']
Y = df['Category']

# ----------------------------------------------------------------------
# B. VERİYİ BÖLME VE ÖZELLİK ÇIKARIMI
# ----------------------------------------------------------------------

# 3. Veriyi Bölme (Stratified Split)
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=0.2,
    random_state=42,
    stratify=Y
)

# 4. Özellik Çıkarımı (Karakter N-Gramları)
# analyzer='char_wb' ile Karakter N-Gramları kullanılır.
vectorizer_char = TfidfVectorizer(
    min_df=1,
    stop_words='english',
    ngram_range=(1, 2),
    lowercase=True
)

# Vektörleyiciyi eğit ve veriyi dönüştür
X_train_features = vectorizer_char.fit_transform(X_train)
X_test_features = vectorizer_char.transform(X_test)

# ----------------------------------------------------------------------
# C. MODELLEME VE DEĞERLENDİRME
# ----------------------------------------------------------------------

# 5. Modeli Eğitme (Linear SVC ile C=1.0)
# yapay zeka deneme yanılma sonunca C=1.0 değeri en iyi dengeyi sağlamıştır.
model = LinearSVC(random_state=42, C=1.0)
model.fit(X_train_features, Y_train)

# 6. Test ve Değerlendirme
y_pred = model.predict(X_test_features)
accuracy = accuracy_score(Y_test, y_pred)
cm = confusion_matrix(Y_test, y_pred)

print(f"Linear SVC: {accuracy * 100:.2f}%")
print(f"Confusion Matrix:\n{cm}")

Linear SVC: 98.30%
Confusion Matrix:
[[963   3]
 [ 16 133]]


## LSVC (Karakter)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC # En iyi sonuç veren model
from sklearn.metrics import accuracy_score, confusion_matrix

# ----------------------------------------------------------------------
# A. VERİ YÜKLEME VE ÖN İŞLEME
# ----------------------------------------------------------------------

# 1. Veriyi Yükleme
try:
    df = pd.read_csv('mail_data.csv')
except UnicodeDecodeError:
    df = pd.read_csv('mail_data.csv', encoding='latin-1')

df.columns = ['Category', 'Message']
df = df[['Category', 'Message']]
df = df.where((pd.notnull(df)), '')

# 2. Etiketleme: ham -> 0, spam -> 1
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

X = df['Message']
Y = df['Category']

# ----------------------------------------------------------------------
# B. VERİYİ BÖLME VE ÖZELLİK ÇIKARIMI
# ----------------------------------------------------------------------

# 3. Veriyi Bölme (Stratified Split)
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=0.2,
    random_state=42,
    stratify=Y
)

# 4. Özellik Çıkarımı (Karakter N-Gramları)
# analyzer='char_wb' ile Karakter N-Gramları kullanılır.
vectorizer_char = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(2, 4), # 2'li, 3'lü ve 4'lü karakter gruplarına bakar
    lowercase=True
)

# Vektörleyiciyi eğit ve veriyi dönüştür
X_train_features = vectorizer_char.fit_transform(X_train)
X_test_features = vectorizer_char.transform(X_test)

# ----------------------------------------------------------------------
# C. MODELLEME VE DEĞERLENDİRME
# ----------------------------------------------------------------------

# 5. Modeli Eğitme (Linear SVC ile C=1.0)
# C=1.0 değeri en iyi dengeyi sağlamıştır.
model = LinearSVC(random_state=42, C=1.0)
model.fit(X_train_features, Y_train)

# 6. Test ve Değerlendirme
y_pred = model.predict(X_test_features)
accuracy = accuracy_score(Y_test, y_pred)
cm = confusion_matrix(Y_test, y_pred)

print(f"NİHAİ MODEL BAŞARISI (Linear SVC + Karakter N-Gram): {accuracy * 100:.2f}%")
print(f"Confusion Matrix:\n{cm}")

NİHAİ MODEL BAŞARISI (Linear SVC + Karakter N-Gram): 99.01%
Confusion Matrix:
[[966   0]
 [ 11 138]]


# Ekstra Test Kısmı (LinearSVC - Karakter Tabanlı Model)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ----------------------------------------------------------------------
# A. MODEL EĞİTİMİ (mail_data.csv KULLANILARAK)
# ----------------------------------------------------------------------
print("--- 1. Model Eğitiliyor (mail_data.csv) ---")

# 1. Eğitim Verisini Yükleme
try:
    df = pd.read_csv('mail_data.csv')
except UnicodeDecodeError:
    df = pd.read_csv('mail_data.csv', encoding='latin-1')

# Sütunları düzenleme
df.columns = ['Category', 'Message']
df = df.where((pd.notnull(df)), '')

# Etiketleme: ham -> 0, spam -> 1
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

X = df['Message']
Y = df['Category']

# Veriyi Bölme
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y
)

# Özellik Çıkarımı (Eğitim verisi üzerinde 'fit' edilir)
vectorizer_char = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(2, 4),
    lowercase=True
)
X_train_features = vectorizer_char.fit_transform(X_train)
X_test_features = vectorizer_char.transform(X_test)

# Modeli Eğitme
model = LinearSVC(random_state=42, C=1.0)
model.fit(X_train_features, Y_train)

# Eğitim Seti Üzerindeki Başarı (İç Test)
train_acc = accuracy_score(Y_test, model.predict(X_test_features))
print(f"Modelin İç Test Başarısı (Validation): %{train_acc * 100:.2f}")


# ----------------------------------------------------------------------
# D. Ekstra test
# ----------------------------------------------------------------------
print("\n" + "="*50)

# Dataset'te olmayan, tamamen uydurma cümleler
my_custom_tests = [
    # Zorlayıcı HAM -- bunları düzgün işaretleyebiliyor.
    "Hey can you send me the free ticket code you promised?",
    "I won the chess tournament! It feels great.",

    # SPAM | yakalayamıyor -- eğitim setinde çok fazla böyle veri olmadığı için.
    "Urgent bank update. Click link to verify.",
    #kesin yakalıyor -- eğitim setinde bolca bunlardan var.
    "You have a package waiting. Pay shipping 2$ here."
    "WINNER! Claim your free prize now!",
    "URGENT! Please call 09061213237 immediately.",
    "Free entry to win 1000 cash txt CASH to 88888"

]

# Tahmin
features = vectorizer_char.transform(my_custom_tests)
preds = model.predict(features)

for text, label in zip(my_custom_tests, preds):
    label_name = "SPAM" if label == 1 else "HAM"
    print(f"Mesaj: {text}\nTahmin: {label_name}\n---")

--- 1. Model Eğitiliyor (mail_data.csv) ---
Modelin İç Test Başarısı (Validation): %99.01

Mesaj: Hey can you send me the free ticket code you promised?
Tahmin: HAM
---
Mesaj: I won the chess tournament! It feels great.
Tahmin: HAM
---
Mesaj: Urgent bank update. Click link to verify.
Tahmin: HAM
---
Mesaj: You have a package waiting. Pay shipping 2$ here.WINNER! Claim your free prize now!
Tahmin: SPAM
---
Mesaj: URGENT! Please call 09061213237 immediately.
Tahmin: SPAM
---
Mesaj: Free entry to win 1000 cash txt CASH to 88888
Tahmin: SPAM
---


## grid search ile daha iyi bir sonuç elde edilebilir. Belki??